# Azure OpenAI Service - Chat on private data using LangChain

Firstly, create a file called `.env` in this folder, and add the following content, obviously with your values:

```
OPENAI_API_KEY=xxxxxx
OPENAI_API_BASE=https://xxxxxxx.openai.azure.com/
```

Then, let's install all dependencies:

In [ ]:
!pip install -r ../requirements.txt

In [1]:
import os
import openai
from dotenv import load_dotenv
from langchain.chat_models import AzureChatOpenAI
from langchain.embeddings import OpenAIEmbeddings

# Load environment variables (set OPENAI_API_KEY and OPENAI_API_BASE in .env)
load_dotenv()

# Configure OpenAI API
openai.api_type = "azure"
openai.api_version = "2023-03-15-preview"
openai.api_base = os.getenv('OPENAI_API_BASE')
openai.api_key = os.getenv("OPENAI_API_KEY")

# Init LLM and embeddings model
llm = AzureChatOpenAI(deployment_name="gpt-35-turbo", openai_api_version="2023-03-15-preview")
embeddings = OpenAIEmbeddings(model="text-embedding-ada-002", chunk_size=1)

First, we load up our documents from the `data` directory:

In [2]:
from langchain.document_loaders import DirectoryLoader
from langchain.document_loaders import TextLoader
from langchain.text_splitter import TokenTextSplitter
from langchain.chains import RetrievalQAWithSourcesChain

loader = DirectoryLoader('../data/qna/', glob="*.txt", loader_cls=TextLoader)

documents = loader.load()
text_splitter = TokenTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(documents)

Next, let's ingest them into FAISS so we can efficiently query our embeddings:

In [3]:
from langchain.vectorstores import FAISS
db = FAISS.from_documents(documents=docs, embedding=embeddings)

Now let's create a chain that can do the whole chat on our embedding database:

In [4]:
from langchain.chains import ConversationalRetrievalChain
from langchain.prompts import PromptTemplate

# Adapt if needed
CONDENSE_QUESTION_PROMPT = PromptTemplate.from_template("""Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question.

Chat History:
{chat_history}
Follow Up Input: {question}
Standalone question:""")

qa = ConversationalRetrievalChain.from_llm(llm=llm,
                                           retriever=db.as_retriever(),
                                           condense_question_prompt=CONDENSE_QUESTION_PROMPT,
                                           return_source_documents=True,
                                           verbose=False)

In [5]:
chat_history = []
query = "what is azure openai service?"
result = qa({"question": query, "chat_history": chat_history})
print(result["answer"])

The Azure OpenAI service is an offering on the Microsoft Azure platform that provides REST API access to OpenAI's powerful language models, including but not limited to GPT-3, Codex, and Embeddings model series. These models can be easily adapted to your specific task, including content generation, summarization, semantic search, and natural language to code translation. In addition, Azure OpenAI offers virtual network support and managed identity via Azure Active Directory. The service is available in East US, South Central US, and West Europe regions. Access to the service is limited as they navigate high demand, upcoming product improvements, and responsible AI, but you can apply for initial access or a production review through their website.


In [6]:
chat_history = [(query, result["answer"])]
query = "Does it support gpt-3, give a long answer"
result = qa({"question": query, "chat_history": chat_history})
print(result["answer"])

Yes, of course! The Azure OpenAI service provides access to powerful language models, including the GPT-3, Codex, and Embeddings model series. These models can be used for a variety of natural language processing tasks, such as content generation, summarization, semantic search, and natural language to code translation.

Users can access the Azure OpenAI service through REST APIs, Python SDK, or a web-based interface in the Azure OpenAI Studio. The service also provides various features and capabilities, including fine-tuning, virtual network support, and managed identity via Azure Active Directory.

Azure OpenAI also offers responsible AI capabilities to prevent the creation of incorrect or harmful content generated by these models. Microsoft has made significant investments to help guard against abuse and unintended harm, which includes requiring well-defined use cases, incorporating Microsoft's principles for responsible AI use, and building content filters to support customers, amo